In [32]:
import requests
import pandas as pd
import datetime
"""
For testing on local machine

The function works by requesting a json of the latest road incidents from the LTA Dynamic Data API. This lambda is queried hourly. After quering the data, 
it checks if the api returns any data. If there is, it converts it into a pandas table, and creates the Date and Time column. It then filters out to keep the data
from the past hourly, and checks again if there is any data. If there is, it converts it into a json table, and is inserted into the DynamoDB table.
"""

# URL and Pulling Data
traffic_url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"
api_key = open("api_key.txt").read()
api_headers = {"AccountKey": api_key, "accept": "application/json"}

request = requests.get(url=traffic_url, headers=api_headers)
print(request)
data = request.json()
dt = datetime.datetime.now()    # + datetime.timedelta(hours = 8)
                                # Only needed in AWS due to time difference

df = pd.DataFrame(data["value"])

<Response [200]>


In [33]:
df = pd.DataFrame(data["value"])
df

,Type,Latitude,Longitude,Message
0,Roadwork,1.302650,103.857574,(29/12)10:26 Roadworks on Victoria Street near...
1,Roadwork,1.284760,103.799223,(29/12)10:25 Roadworks on AYE (towards Tuas) a...
2,Roadwork,1.275462,103.846211,(29/12)10:25 Roadworks on Anson Road (towards ...
3,Roadwork,1.282797,103.801329,(29/12)10:24 Roadworks on Alexandra Road (towa...
4,Roadwork,1.304777,103.859835,(29/12)10:23 Roadworks on Victoria Street (tow...
...,...,...,...,...
72,Roadwork,1.318804,103.911334,(29/12)9:35 Roadworks on Changi Road (toward C...
73,Roadwork,1.356743,103.715544,(29/12)9:30 Roadworks on PIE (towards Changi A...
74,Accident,1.423280,103.861517,(29/12)9:14 Accident on Yishun Avenue 1 (towar...
75,Roadwork,1.318092,103.747710,(29/12)9:13 Roadworks on West Coast Road (towa...


In [34]:
len(df) # Add line to break

if (len(df) > 0):
    print()
    # Continue with code

In [35]:
# Extract Date and Time out of the Message to keep the main message only

date_regex = "([0-9]{1,2}/[0-9]{1,2})"
df["Date"] = df["Message"].str.extract(pat = date_regex) + "/" + str(dt.year)

time_regex = "([0-9]{1,2}:[0-9]{1,2})"
df["Time"] = df["Message"].str.extract(pat = time_regex)

message_regex = "\d\s(.*$)"
df["Message"] = df["Message"].str.extract(pat = message_regex)

In [36]:
df

,Type,Latitude,Longitude,Message,Date,Time
0,Roadwork,1.302650,103.857574,Roadworks on Victoria Street near Arab Street/...,29/12/2022,10:26
1,Roadwork,1.284760,103.799223,Roadworks on AYE (towards Tuas) at Alexandra R...,29/12/2022,10:25
2,Roadwork,1.275462,103.846211,Roadworks on Anson Road (towards Prince Edward...,29/12/2022,10:25
3,Roadwork,1.282797,103.801329,Roadworks on Alexandra Road (towards Bury Road...,29/12/2022,10:24
4,Roadwork,1.304777,103.859835,Roadworks on Victoria Street (towards Jalan Su...,29/12/2022,10:23
...,...,...,...,...,...,...
72,Roadwork,1.318804,103.911334,Roadworks on Changi Road (toward City) after F...,29/12/2022,9:35
73,Roadwork,1.356743,103.715544,Roadworks on PIE (towards Changi Airport) afte...,29/12/2022,9:30
74,Accident,1.423280,103.861517,Accident on Yishun Avenue 1 (towards Seletar W...,29/12/2022,9:14
75,Roadwork,1.318092,103.747710,Roadworks on West Coast Road (towards Jurong T...,29/12/2022,9:13


In [37]:
# Filtering to keep only previous hour of data
if dt.hour >= 1:
    prev_hour = str(dt.hour - 1)
    df_boolean = df["Time"].str.match(pat = (prev_hour + ":"))
else:
    prev_hour = "23"
    df_boolean = df["Time"].str.match(pat = "23:")

In [38]:
df_tocloud = df[df_boolean].reset_index(drop = True)
df_tocloud

,Type,Latitude,Longitude,Message,Date,Time
0,Roadwork,1.307262,103.818830,Roadworks on Cluny Road (towards Taman Serasi)...,29/12/2022,9:56
1,Roadwork,1.306719,103.818236,Roadworks on Holland Road (towards Cluny Road)...,29/12/2022,9:55
2,Roadwork,1.307509,103.818836,Roadworks on Cluny Road (towards Holland Road)...,29/12/2022,9:54
3,Roadwork,1.306407,103.913769,Roadworks on Marine Parade Road (towards Telok...,29/12/2022,9:53
4,Roadwork,1.311152,103.802946,Roadworks on Holland Road (towards Farrer Road...,29/12/2022,9:53
5,Roadwork,1.300223,103.801041,Roadworks on Queensway (towards AYE) after Hol...,29/12/2022,9:53
6,Roadwork,1.306790,103.915026,Roadworks on Marine Parade Road (towards Marin...,29/12/2022,9:53
7,Roadwork,1.399990,103.773798,Roadworks on BKE (towards PIE) after Mandai Rd...,29/12/2022,9:51
8,Roadwork,1.273055,103.800567,Roadworks on Pasir Panjang Road (towards Tuas)...,29/12/2022,9:51
9,Roadwork,1.423213,103.771175,Roadworks on BKE (towards Woodlands) after SLE...,29/12/2022,9:50


In [21]:
df_tocloud_key = str(dt.date()) + " " + prev_hour
df_tocloud_key

'2022-12-28 16'

In [31]:
print(str(dt.date() - datetime.timedelta(days=1)) + " " + prev_hour)

print(str(dt.date()) + " " + prev_hour)

2022-12-27 16
2022-12-28 16


## For Testing and Subsequent Wring

In [130]:
# Converting to JSON for Writing to Cloud
df_tocloud = df_tocloud.to_json()


# Reading from Cloud
import json
df_raw = json.loads(df_tocloud)
df_local = pd.DataFrame(df_raw)
df_local

,Type,Latitude,Longitude,Message,Date,Time
0,Roadwork,1.290572,103.842833,Roadworks in CTE Tunnel (towards AYE) before H...,27/12/2022,23:57
1,Roadwork,1.306701,103.840467,Roadworks in CTE Tunnel (towards AYE) before O...,27/12/2022,23:55
2,Roadwork,1.396506,103.847941,Roadworks on SLE (towards BKE) after TPE Entra...,27/12/2022,23:54
3,Roadwork,1.326460,103.870421,Roadworks on PIE (towards Tuas) after KPE Exit...,27/12/2022,23:53
4,Roadwork,1.310076,103.870834,Roadworks on Geylang Road (toward City) after ...,27/12/2022,23:50
5,Roadwork,1.328920,103.868459,Roadworks in Woodsville Tunnel (towards Upper ...,27/12/2022,23:48
6,Roadwork,1.293474,103.844128,Roadworks in CTE Tunnel (towards AYE) before H...,27/12/2022,23:45
7,Roadwork,1.353673,103.727579,Roadworks on PIE (towards Tuas) at Jurong East...,27/12/2022,23:45
8,Roadwork,1.341450,103.749688,Roadworks on PIE (towards Changi Airport) at T...,27/12/2022,23:38
9,Roadwork,1.396502,103.904031,Roadworks on TPE (towards SLE) after Punggol R...,27/12/2022,23:28
